# Opentrons Protocol for AFB Epistasis Mutant Cloning

First we will import python API (Application Programming Interface) into our script so that we can develop a protol that can be understood by the robot


In [6]:
!pip install --upgrade opentrons

Requirement already up-to-date: opentrons in c:\users\jonbr\anaconda3\lib\site-packages (3.21.2)


Metadata is a dictionary of data that is read by the server and returned to the opentrons app. give yourself credit.
you are however required to specify the 'apiLevel' here

In [5]:
from opentrons import protocol_api
metadata = {
    'protocolName': 'AFB Epistasis Mutations Protocol',
    'author': 'John Bryant <jbryant2@vt.edu>',
    'description': 'Protocol for performing PCR reactions and Plasmid assembly for TIR1 and AFB mutants',
    'apiLevel': '2.6'
}
print(metadata)

{'protocolName': 'AFB Epistasis Mutations Protocol', 'author': 'John Bryant <jbryant2@vt.edu>', 'description': 'Protocol for performing PCR reactions and Plasmid assembly for TIR1 and AFB mutants', 'apiLevel': '2.6'}


Protocols are structured around a function called run(protocol). The function has to be name "run" and must take exactly one mandatory argument ("protocol"). function run is a container for the code that defines your protocol. 

protocol context has 2 responsibilities:
1)remember, track, and check robot's state
2)expose the functions that make the robot execute actions.

In [34]:
#calculation of how much H2O to add to primer dilutions
A=10 #this is how much stock primer you add
A2=400 # this is the volume you bring it up to
X=((A)*100)/A2 #diluted primer concentration
print('initial diluted primer concentration is:',str(X))
prmh20 = A2-A #this is how much h20 you need to add to
print('add',str(prmh20),'uL h2O')


initial diluted primer concentration is: 2.5
add 390 uL h2O


In [37]:
#concentration of templates
tempconc1= 237 #concentration of concentrated template
B = 1 #uL of stock template to start with
C = 12.5 #desired ng/uL of template (your diluted template concentration)
h20totemp=((B*tempconc1)/C)-B
print('add',str(h20totemp),'uL water to',str(B),'uL stock template')

add 17.96 uL water to 1 uL stock template


now after initial dilutions we have primers at 2.5uM/L and templates at 12.5ng/uL
in the pcr cocktain it is 25 uL total with 0.5ng of each template and 0.1 uM primers


In [39]:
pcrprim1=1 #amount in microliters to add to pcr reaction
pcrtotvol=25 #total vol of pcr rxn in uL
finalprmconc=(X*pcrprim1)/pcrtotvol #should be 0.1
print('final primer concentration is:',str(finalprmconc))

final primer concentration is: 0.1


In [42]:
template=1 #amount in microliters to add to pcr reaction
pcrtotvol=25 #total vol of pcr rxn in uL
(template*C)/25

0.5

In [48]:
#how much water do we add to this pcr?
Q5=12.5
amth20=(pcrtotvol-Q5-pcrprim1-pcrprim1-pcrprim1-pcrprim1-template)#each tube has four primers and one template
print('add',str(amth20), 'h2o to reaction')

add 7.5 h2o to reaction


In [12]:





def run(protocol: protocol_api.ProtocolContext):
    
    #labware:
    tiprack1 = protocol.load_labware('opentrons_96_tiprack_300ul', '1')
    tiprack2 = protocol.load_labware('opentrons_96_tiprack_20ul','2')
    tuberack1 = protocol.load_labware('opentrons_24_tuberack_generic_2ml_screwcap','3') #holds stock primers and templates
    bigtuberack = protocol.load_labware('opentrons_15_tuberack_falcon_15ml_conical','4') #holds molec bio grad H2O
    tuberack2 = protocol.load_labware('opentrons_24_tuberack_nest_1.5ml_snapcap','5') # holds dilute primers and templates
    
    tc_mod = protocol.load_module('Thermocycler Module')
    plate = tc_mod.load_labware('nest_96_wellplate_100ul_pcr_full_skirt')
    
    #can put dpnI digestion on either place 6 or 9
    
    
    #pipettes
    right_pipette = protocol.load_instrument('p300_single','right',tipracks=[tiprack1])
    left_pipette = protocol.load_instrument('p20_single','left',tipracks=[tiprack2])
    
    #commands
    
    #put water in for diluted primers (using stkprmh20 variable defined earlier)
    p200.distribute(stkprmh20, bigtuberack.wells('C5'), tuberack2.wells('A1','A2','A3','A4','A5','A6','B1','B2','B3','B4','B5','B6','C1','C2','C3','C4','C5','C6','D1','D2','D3'))  # dilutent
 
    # loop through each tuberack well to dilute stock primers
    for i in range(21):

      # save the source well and destination column to variables
      source = tuberack1.wells(i)
      destination = tuberack2.wells(i)

      # transfer 10uL of source to first well in column (using A variable defined earlier)
      left_pipette.transfer(A, source, destination)

      # dilute the sample down the column (using A variable defined earlier)
      left_pipette.transfer(
        A, column.wells('1', to='6'), column.wells('2', to='4'),
        mix_after=(3, 25))
    
    
    #need to loop through and dilute both of the templates (using variable B defined above)
    p200.distribute(h20totemp, bigtuberack.wells('C5'), tuberack2.wells('D4','D5'))  # dilutent
 
    
    left_pipette.pick_up_tip()
    left_pipette.aspirate(B,tuberack1['D5'])
    left_pipette.dispense(B,tuberack2['D5'])
    left_pipette.drop_tip()
    #########################################################################################
    
    #time to start filling up the pcr tubes
    
    #first add appropriate amount of water
    #put water in for diluted primers (using stkprmh20 variable defined earlier)
    p200.distribute(amth20, bigtuberack.wells('C5'), plate.wells('A1','A2','A3','A4','A5','A6','A7','A8','A9'] # dilutent
 
    
    #move diluted primers into pcr tubes how do i do this with sets of four primers going into each pcr tube
    # loop through each tuberack well to dilute diluted primers
    for i in range(21):

      # save the source well and destination column to variables
      source = tuberack2.wells(i)
      destination = tuberack2.wells(i)

      # transfer 10uL of source to first well in column (using A variable defined earlier)
      left_pipette.transfer(A, source, destination)

      # dilute the sample down the column (using A variable defined earlier)
      left_pipette.transfer(
        A, column.wells('1', to='6'), column.wells('2', to='4'),
        mix_after=(3, 25))
    
      #move diluted templates into pcr tubes - this should probable be done with an if else statement
                                                                 
    
   
    #######################################################################################################
                                                                 
    #set up pcr reaction in the thermocycler module
    
    #open lid
    tc_mod.open_lid()
    
    #close lid
    tc_mod.close_lid()
    
    #set lid temperature
    tc_mod.set_lid_temperature(100)
    
    #set block temp profile
   
    tc_mod.execute_profile(98, hold_time_seconds=30, block_max_volume=50)

    profile = [
    {'temperature': 98, 'hold_time_seconds': 10},
    {'temperature': 65, 'hold_time_seconds': 30},
    {'temperature': 72, }]

    tc_mod.execute_profile(steps=profile, repetitions=34, block_max_volume=50)
    
    tc_mod.execute_profile(72, hold_time_minutes=5, block_max_volume=50)
    
    ################################################################################
    #add DpnI for digestion
    #add DpnI to each PCR product
    
    

    

In [11]:
#calculation of how much H2O to add to primer dilutions
A=1 #this is how much stock primer you add
X=((A)*100)/1
stkprmh20 = X-A



99.0


In [19]:
each pcr tube has 25 uL




17.96
